# Analysis of Changes in Biodiveristy Intactness Index (BII) in the Phoenix Metro Area

Author: Rachel Swick

Date: December 5, 2024

Github Link: https://github.com/rfswick/bii-changes-maricopa-county.git

## Purpose

To understand changes in BII in Maricopa County, which contains the Phoenix metro area, by creating visualizations showing BII in Maricopa County.

## Highlights

- 
- 
- 

## About the Data

## References


## Import Libraries

In [3]:
# Load libraries
import contextily
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import rioxarray as rioxr
from pystac_client import Client
import planetary_computer
from IPython.display import Image

## Load Data

### Biodiversity Intactness Index (BII) Time Series Data



In [60]:
# Load BII data for 2017 and 2020
# Access MPC catalog
catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
)

# Maricopa county bounding box
bbox = {
        "type": "Polygon",
        "coordinates":[
        [
            [-112.826843, 32.974108],  # lower left corner (longitude, latitude)
            [-111.184387, 32.974108],  # lower right corner
            [-111.184387, 33.863574],  # upper right corner
            [-112.826843, 33.863574],  # upper left corner
            [-112.826843, 32.974108]   # closing the polygon (same as start point)
        ]
    ],
}

# Search catalog and retrieve search items
items = catalog.search(
        collections = ['io-biodiversity'],
        intersects = bbox).item_collection()

# Four items in the catalog match the search
# Determine which items in the list contain 2017 and 2020 bii data
list(items)

[<Item id=bii_2020_34.74464974521749_-115.38597824385106_cog>,
 <Item id=bii_2019_34.74464974521749_-115.38597824385106_cog>,
 <Item id=bii_2018_34.74464974521749_-115.38597824385106_cog>,
 <Item id=bii_2017_34.74464974521749_-115.38597824385106_cog>]

In [67]:
# Select the 2017 and 2020 items in the catalog search
year_17 = items[3]
year_20 = items[0]

# Print `year_17` ID and assets
print('ID:' , year_17.id)
for key in year_17.assets.keys():
    print(key, '--', year_17.assets[key].title)

# Print `year_20` ID and assets
print('ID:' , year_20.id)
for key in year_20.assets.keys():
    print(key, '--', year_20.assets[key].title)

ID: bii_2017_34.74464974521749_-115.38597824385106_cog
data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview
ID: bii_2020_34.74464974521749_-115.38597824385106_cog
data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [69]:
# Load 2017 and 2020 BII data
bii_17 = rioxr.open_rasterio(year_17.assets['data'].href)
bii_20 = rioxr.open_rasterio(year_20.assets['data'].href)

### Phoenix Subdivision Shapefile

In [85]:
# Load Arizona county subdivisions
arizona_subdivisions = gpd.read_file(os.path.join('data', 'tl_2022_04_cousub', 'tl_2022_04_cousub.shp'))

# Filter to Phoenix subdivision
phoenix_subdivision = arizona_subdivisions[arizona_subdivisions['NAME'] == 'Phoenix']

## Data Exploration



In [70]:
# Basic description of `io-biodiversity` collection
catalog.get_child('io-biodiversity')

<CollectionClient id=io-biodiversity>